### initialise things

In [9]:
import importlib
import yield_curve as zCurve
importlib.reload(zCurve)
import instruments as inst
importlib.reload(inst)
import pandas as pd
import tabulate as tb

### Start by setting up a zero curve for the purposes of running PV calcs on the instruments

In [10]:
# create an instance of the ZeroCurve class
zc = zCurve.ZeroCurve()

# add some zero rates to the curve
zc.add_zero_rate(1, 0.05)
zc.add_zero_rate(2, 0.06)
zc.add_zero_rate(3, 0.07)


### Set up a simple cash_flows object containing to cashflows manually added

In [11]:
# create an instance of the CashFlows class called my_cash_flows
my_cash_flows = inst.CashFlows()

# add a cash flow of 1000 at time 1
my_cash_flows.add_cash_flow(1, 1000)

# add a cash flow of 2000 at time 2
my_cash_flows.add_cash_flow(2.5, 1500)

print(my_cash_flows.get_cash_flows())
print(my_cash_flows.get_maturities())
print(my_cash_flows.get_amounts())
print(zc.npv(my_cash_flows))

# create a new dataframe called cash_flows_df to store the cash flows
cash_flows_df = pd.DataFrame(data={'maturity': my_cash_flows.get_maturities(), 'amount': my_cash_flows.get_amounts()})
# set the index of the dataframe to be the maturity
cash_flows_df.set_index('maturity', inplace=True)
cash_flows_df





[(1, 1000), (2.5, 1500)]
[1, 2.5]
[1000, 1500]
2223.069980632588


,amount
maturity,
1.0,1000
2.5,1500


### Set up and value a bank_bill

In [12]:
# create an instance of the bank_bill class called my_bank_bill
my_bank_bill = inst.Bank_bill()
my_bank_bill.set_ytm(0.06)
my_bank_bill.set_cash_flows()

bank_bill_cashflows= my_bank_bill.get_cash_flows()
print(bank_bill_cashflows)
print(zc.npv(my_bank_bill))

# create a new dataframe called cash_flows_df to store the cash flows
bill_cash_flows_df = pd.DataFrame(data={'Maturity': my_bank_bill.get_maturities(), 'Amount': my_bank_bill.get_amounts()})
# set the index of the dataframe to be the maturity
bill_cash_flows_df.set_index('Maturity', inplace=True)
bill_cash_flows_df

[(0, -98.52216748768474), (0.25, 100)]
-0.5175974741957248


,Amount
Maturity,
0.00,-98.522167
0.25,100.000000


### Set up and value a bond

In [13]:
# create an instance of the bond class called my_bond
my_bond = inst.Bond()
my_bond.set_maturity(1)
my_bond.set_coupon(0.05)
my_bond.set_frequency(2)
my_bond.set_face_value(100)
my_bond.set_ytm(0.05)

my_bond.set_cash_flows()
bond_cashflows = my_bond.get_cash_flows()
print("The cashflows of the bond are : " + str(bond_cashflows))
print("The PV of the bond is : " + str(zc.npv(my_bond)))
print("")

# create a new dataframe called cash_flows_df to store the cash flows
bond_cash_flows_df = pd.DataFrame(data={'Maturity': my_bond.get_maturities(), 'Amount': my_bond.get_amounts()})
# set the index of the dataframe to be the maturity
bond_cash_flows_df.set_index('Maturity', inplace=True)

print("Here are the cashflows of the bond in a dataframe : ")
print(bond_cash_flows_df.to_string())
print("")
print("Here are the cashflows of the bond in a table using the tabulate library: ")
print(tb.tabulate(bond_cash_flows_df, headers='keys', tablefmt='psql'))

test_price = (my_bond.coupon/my_bond.frequency*my_bond.face_value)*(1-(1+my_bond.ytm/my_bond.frequency)**(-my_bond.maturity*my_bond.frequency))/(my_bond.ytm/my_bond.frequency) \
          + my_bond.face_value/((1 + my_bond.ytm/my_bond.frequency)**(my_bond.maturity*my_bond.frequency))
print(test_price)

The cashflows of the bond are : [(0, -99.99999999999999), (0.5, 2.5), (1, 102.5)]
The PV of the bond is : -3.069312799655563

Here are the cashflows of the bond in a dataframe : 
          Amount
Maturity        
0.0       -100.0
0.5          2.5
1.0        102.5

Here are the cashflows of the bond in a table using the tabulate library: 
+------------+----------+
|   Maturity |   Amount |
|------------+----------|
|        0   |   -100   |
|        0.5 |      2.5 |
|        1   |    102.5 |
+------------+----------+
99.99999999999999


### Set up a portfolio containing some bank bills and bonds
The portfolio includes the bond and bill set up above as well as a new one of each.

In [14]:
# create an instance of the portfolio class called my_portfolio
my_portfolio = inst.Portfolio()

# create another instance of the bond class called my_bond_2
my_bond_2 = inst.Bond()
my_bond_2.set_maturity(2)
my_bond_2.set_coupon(0.08)
my_bond_2.set_frequency(4)
my_bond_2.set_face_value(1000)
my_bond_2.set_ytm(0.04)
my_bond_2.set_cash_flows()

# add the bonds to the portfolio
my_portfolio.add_bond(my_bond_2)
my_portfolio.add_bond(my_bond)

print("The bonds in the portfolio are : " + str(my_portfolio.get_bonds()))
bond_cashflows = my_bond_2.get_cash_flows()
print("The cashflows of my_bond_2 are : " + str(bond_cashflows))
print("The PV of my_bond_2 is : " + str(zc.npv(my_bond)))
print("")

# create an instance of the bank_bill class called bank_bill_2
bank_bill_2 = inst.Bank_bill()
bank_bill_2.set_maturity(.5)
bank_bill_2.set_ytm(0.07)
bank_bill_2.set_cash_flows()

my_portfolio.add_bank_bill(bank_bill_2)
my_portfolio.add_bank_bill(my_bank_bill)

my_portfolio.set_cash_flows()
portfolio_cashflows = my_portfolio.get_cash_flows()
print("The cashflows of the portfolio are : ")

portfolio_cashflows_df = pd.DataFrame(data={'Maturity': my_portfolio.get_maturities(), 'Amount': my_portfolio.get_amounts()})
print(portfolio_cashflows_df.to_string())

print("The PV of the portfolio is : " + str(zc.npv(my_portfolio)))
print("")






The bonds in the portfolio are : [<instruments.Bond object at 0x7e05818d74a0>, <instruments.Bond object at 0x7e05817ea1e0>]
The cashflows of my_bond_2 are : [(0, -1076.5167775176878), (0.25, 20.0), (0.5, 20.0), (0.75, 20.0), (1.0, 20.0), (1.25, 20.0), (1.5, 20.0), (1.75, 20.0), (2, 1020.0)]
The PV of my_bond_2 is : -3.069312799655563

The cashflows of the portfolio are : 
    Maturity       Amount
0       0.00 -1076.516778
1       0.25    20.000000
2       0.50    20.000000
3       0.75    20.000000
4       1.00    20.000000
5       1.25    20.000000
6       1.50    20.000000
7       1.75    20.000000
8       2.00  1020.000000
9       0.00  -100.000000
10      0.50     2.500000
11      1.00   102.500000
12      0.00   -96.618357
13      0.50   100.000000
14      0.00   -98.522167
15      0.25   100.000000
The PV of the portfolio is : -3320.1680841742987

